In [3]:
import av
import cv2
import numpy as np
import pandas as pd

In [15]:
images_av = []
images_cv2 = []


In [16]:
%%timeit
container = av.open("./videos/WUzgd7C1pWA.mp4")
for frame in container.decode(video=0):
    images_av.append(frame.to_rgb().to_ndarray())



455 ms ± 29.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
cap = cv2.VideoCapture("./videos/WUzgd7C1pWA.mp4")
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret is True:
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        images_cv2.append(frame)
    else:
        break
cap.release()

188 ms ± 24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
images_av = []
images_cv2 = []
container = av.open("./videos/WUzgd7C1pWA.mp4")
for frame in container.decode(video=0):
    images_av.append(frame.to_rgb().to_ndarray())

cap = cv2.VideoCapture("./videos/WUzgd7C1pWA.mp4")
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret is True:
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        images_cv2.append(frame)
    else:
        break
cap.release()

In [19]:

print(len(images_av), len(images_cv2))

327 327


In [1]:
import torchvision
torchvision.set_video_backend("video_reader")

In [2]:
vframes, _, _ = torchvision.io.read_video("./videos/WUzgd7C1pWA.mp4")
vframes.size()

/Users/bjuncek/bin/conda/envs/video_benchmark/lib/python3.7/site-packages/torchvision-0.8.0a0+9b80465-py3.7-macosx-10.9-x86_64.egg/torchvision/io/_video_opt.py:479: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "


torch.Size([327, 256, 340, 3])

# Simple measurement of python overhead

In [1]:
# first, import torch
import torch

In [2]:
%%timeit
# torch multitensor
a = torch.zeros(1000, 3, 112, 112)

70.4 ms ± 5.56 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
%%timeit
# for loop with array allocation
test = []
for i in range(1000):
    test.append(torch.zeros(3, 112, 112))

93 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%timeit
# for loop with array allocation and stacking
test = []
for i in range(1000):
    test.append(torch.zeros(3, 112, 112))
a = torch.stack(test)


179 ms ± 4.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Exploring DECORD

With GPU and CPU support (or at least I hope so)

In [15]:
from decord import VideoReader
from decord import cpu
import numpy as np
vr = VideoReader('videos/SOX5yA1l24A.mp4', ctx=cpu(0))
# a file like object works as well, for in-memory decoding
nf = len(vr)
# 1. the simplest way is to directly access frames


In [24]:
%%timeit
frames = []
for i in range(nf):
    # the video reader will handle seeking and skipping in the most efficient manner
    frames.append(vr[i])


76.6 ms ± 349 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%%timeit
frames = vr.get_batch(range(nf))

100 ms ± 812 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
[0::nf]

SyntaxError: invalid syntax (<ipython-input-10-64743ef96512>, line 1)